In [54]:
import pandas as pd 
import os 
import numpy as np 
import re
import string
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import math
from selenium.common.exceptions import NoSuchElementException

In [55]:
base_url = r"https://www.myneta.info"
html = urlopen(base_url)
soup = BeautifulSoup(html, 'lxml')

### Scrape data on all candidates

In [5]:
columns = ['Name', 'PAN Given(Y or N)', 'Age', 'Gender', 'Party Code', 'Criminal Cases', 'Number of Cases', 
           'Serious IPC Counts', 'IPC Details*', 'Education Level', 'Movable Assets', 
           'Immovable Assets', 'Total Assets', 'Total Liabilities']
num = len(columns)
dic = {}
count = 0
for link in soup.findAll('a'):
    state = link.text
    url = link.get('href').replace(' ', '%20')
    if 'state_assembly' not in url:
        continue
    html = urlopen(url)
    soup1 = BeautifulSoup(html, 'lxml')
    for l in soup1.findAll('a'):
        u = l.get('href')
        if 'state_assembly' in u:
            continue
        if 'winners' in u.lower():
            print("Winners", u)
        elif 'recontestAssetsComparison' in u:
            print("Recontest", u)
        elif u[0] == '/' and u[-1] == '/' and len(u) > 1:
            print("All candidates ", u)
            html = urlopen(base_url + u)
            soup2 = BeautifulSoup(html, 'lxml')
            for item in soup2.findAll('a'):
                lin = str(item.get('href'))
                constituency = item.text
                if 'constituency_id' not in lin:
                    continue
                conlink = ''
                con_id = lin[lin.rfind('=')+1:]
                if 'myneta' in lin:
                    conlink = lin
                else:
                    conlink = base_url + u  + lin
                html = urlopen(conlink)
                soup3 = BeautifulSoup(html, 'lxml')
                for it in soup3.findAll('a'):
                    t = str(it.get('href'))
                    if 'comparisonchart' not in t:
                        continue
                    html = urlopen(base_url + u + t)
                    soup4 = BeautifulSoup(html, 'lxml')
                    table_container = soup4.find_all('table')
                    soup5 = BeautifulSoup(str(table_container), 'html.parser')
                    first = 1
                    for each in soup5.findAll('tr'):
                        if first == 1:
                            first = 0
                            continue
                        else:
                            c = 1
                            name = each.find('a').text
                            name_link = each.find('a').get('href')
                            name_id = name_link[name_link.rfind('=')+1:]
                            dic[count] = {'state': state, 'constituency': constituency, 
                                          'constituency_id': con_id, 'candidate': name, 
                                          'candidate_id': name_id, 'year': u }
                            for content in each.findAll('td'):
                                dic[count][columns[c]] = content.text
                                c+=1
                                if c == num:
                                    c = 1
                            count+=1
    print(state, " done")

Winners /andhrapradesh2019/index.php?action=show_winners&sort=default
All candidates  /andhrapradesh2019/
Recontest /andhrapradesh2019/index.php?action=recontestAssetsComparison
Winners /andhra2014/index.php?action=show_winners&sort=default
All candidates  /andhra2014/
Winners /andhra2014/index.php?action=showWinnersExpense&sortExp=default
Recontest /andhra2014/index.php?action=recontestAssetsComparison
Winners /ap09/index.php?action=show_winners&sort=default
All candidates  /ap09/
All candidates  /andhraPradesh2004/
Andhra Pradesh  done
Winners /ArunachalPradesh2019/index.php?action=show_winners&sort=default
All candidates  /ArunachalPradesh2019/
Recontest /ArunachalPradesh2019/index.php?action=recontestAssetsComparison
Winners /arunachal2014/index.php?action=show_winners&sort=default
All candidates  /arunachal2014/
Winners /arunachal2014/index.php?action=showWinnersExpense&sortExp=default
Recontest /arunachal2014/index.php?action=recontestAssetsComparison
Winners /an2009/index.php?ac

Winners /kerala2016/index.php?action=showWinnersExpense&sortExp=default
Recontest /kerala2016/index.php?action=recontestAssetsComparison
Winners /kerala2011/index.php?action=show_winners&sort=default
All candidates  /kerala2011/
Winners /kerala2011/index.php?action=showWinnersExpense&sortExp=default
Recontest /kerala2011/index.php?action=recontestAssetsComparison
Winners /ker2006/index.php?action=show_winners&sort=default
All candidates  /ker2006/
Kerala  done
Winners /madhyapradesh2018/index.php?action=show_winners&sort=default
All candidates  /madhyapradesh2018/
Winners /madhyapradesh2018/index.php?action=showWinnersExpense&sortExp=default
Recontest /madhyapradesh2018/index.php?action=recontestAssetsComparison
Winners /mp2013/index.php?action=show_winners&sort=default
All candidates  /mp2013/
Winners /mp2013/index.php?action=showWinnersExpense&sortExp=default
Recontest /mp2013/index.php?action=recontestAssetsComparison
Winners /2008mp/index.php?action=show_winners&sort=default
All ca

Winners /telangana2014/index.php?action=show_winners&sort=default
All candidates  /telangana2014/
Winners /telangana2014/index.php?action=showWinnersExpense&sortExp=default
Recontest /telangana2014/index.php?action=recontestAssetsComparison
Telangana  done
Winners /tripura2018/index.php?action=show_winners&sort=default
All candidates  /tripura2018/
Winners /tripura2018/index.php?action=showWinnersExpense&sortExp=default
Recontest /tripura2018/index.php?action=recontestAssetsComparison
Winners /tripura2013/index.php?action=show_winners&sort=default
All candidates  /tripura2013/
Winners /tripura2013/index.php?action=showWinnersExpense&sortExp=default
Recontest /tripura2013/index.php?action=recontestAssetsComparison
Winners /tripura2008/index.php?action=show_winners&sort=default
All candidates  /tripura2008/
Tripura  done
Winners /uttarakhand2017/index.php?action=show_winners&sort=default
All candidates  /uttarakhand2017/
Winners /uttarakhand2017/index.php?action=showWinnersExpense&sortEx

In [6]:
df = pd.DataFrame.from_dict(dic).T
df

,state,constituency,constituency_id,candidate,candidate_id,year,PAN Given(Y or N),Age,Gender,Party Code,Criminal Cases,Number of Cases,Serious IPC Counts,IPC Details*,Education Level,Movable Assets,Immovable Assets,Total Assets,Total Liabilities
0,Andhra Pradesh,ANANTAPUR URBAN,1018,Amarnath Jangati,5177,/andhrapradesh2019/,Y,62,M,BJP,No,0,0,,12th Pass,"11,18,454~ 11 Lacs+","2,10,00,000~ 2 Crore+","2,21,18,454~ 2 Crore+","62,64,388~ 62 Lacs+"
1,Andhra Pradesh,ANANTAPUR URBAN,1018,Anantha Venkatarami Reddy,5022,/andhrapradesh2019/,Y,62,M,YSRCP,No,0,0,,Post Graduate,"1,20,66,000~ 1 Crore+","7,41,00,000~ 7 Crore+","8,61,66,000~ 8 Crore+","18,88,900~ 18 Lacs+"
2,Andhra Pradesh,ANANTAPUR URBAN,1018,B. Shivaiah,6113,/andhrapradesh2019/,Y,40,M,Nava Samaj Party,No,0,0,,12th Pass,"80,000~ 80 Thou+",0~,"80,000~ 80 Thou+",0~
3,Andhra Pradesh,ANANTAPUR URBAN,1018,D. Raghavendra,6109,/andhrapradesh2019/,N,40,M,SUCI(C),No,0,0,,Graduate,"56,488~ 56 Thou+",0~,"56,488~ 56 Thou+",0~
4,Andhra Pradesh,ANANTAPUR URBAN,1018,G. Nagaraju,5023,/andhrapradesh2019/,Y,47,M,INC,Yes,2,2,"Statements conducing to public mischief(505),...",Post Graduate,"14,51,348~ 14 Lacs+","61,58,200~ 61 Lacs+","76,09,548~ 76 Lacs+","36,00,000~ 36 Lacs+"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132803,West Bengal,Kaliaganj,265,PRAMATHA NATH RAY,530,/wb2006/,N,63,M,INC,No,0,0,,Post Graduate,"9,60,479~ 9 Lacs+","5,00,000~ 5 Lacs+","14,60,479~ 14 Lacs+",0~
132804,West Bengal,Karandighi,263,GOKUL ROY,525,/wb2006/,Y,61,M,AIFB,No,0,0,,Graduate,"6,30,193~ 6 Lacs+","4,60,000~ 4 Lacs+","10,90,193~ 10 Lacs+",0~
132805,West Bengal,Karandighi,263,HAZI SAJJAD HUSSAIN,526,/wb2006/,N,81,M,INC,Yes,1,0,"Punishment for Rioting(147), voluntarily cau...",8th Pass,"65,699~ 65 Thou+","35,92,000~ 35 Lacs+","36,57,699~ 36 Lacs+","60,186~ 60 Thou+"
132806,West Bengal,Raiganj,264,CHITTA RANJAN RAY,527,/wb2006/,Y,63,M,INC,No,0,0,,Graduate Professional,"3,94,128~ 3 Lacs+","5,60,000~ 5 Lacs+","9,54,128~ 9 Lacs+","72,110~ 72 Thou+"


In [7]:
df.to_csv(r"C:\Users\nikhi\Desktop\ADR Politicians\candidates_state_elections.csv")

### Scrape data on winners

In [21]:
columns = ['Liabilities', 'Candidate', 'Constituency', 'Party', 'Criminal Case', 'Education', 'Total Assets']
num = len(columns)
dic1 = {}
count = 0
for link in soup.findAll('a'):
    state = link.text
    url = link.get('href').replace(' ', '%20')
    if 'state_assembly' not in url:
        continue
    html = urlopen(url)
    soup1 = BeautifulSoup(html, 'lxml')
    for l in soup1.findAll('a'):
        u = l.get('href')
        if 'state_assembly' in u:
            continue
        if '=show_winners' in u.lower():
            print("Winners", u)
            html = urlopen(base_url + u)
            soup2 = BeautifulSoup(html, 'lxml')
            try:
                table_container = soup2.find_all('table')[2]
            except:
                continue
            soup3 = BeautifulSoup(str(table_container), 'html.parser')
            first = 1
            for each in soup3.findAll('tr'):
                if first == 1:
                    first = 0
                    continue
                else:
                    dic1[count] = {'state': state, 'year': u[:u.rfind('/')] }
                    c = 0
                    for content in each.findAll('td'):
                        if content.find('a') is not None:
                            name_link = content.find('a').get('href')
                            name_id = name_link[name_link.rfind('=')+1:]
                            dic1[count]['candidate_id'] = name_id
                        dic1[count][columns[c]] = content.text
                        c+=1
                        if c == num:
                            c = 0
                    count+=1
        elif 'recontestAssetsComparison' in u:
            print("Recontest", u)
        elif u[0] == '/' and u[-1] == '/' and len(u) > 1:
            print("All candidates ", u)
    print(state, " done")

Winners /andhrapradesh2019/index.php?action=show_winners&sort=default
All candidates  /andhrapradesh2019/
Recontest /andhrapradesh2019/index.php?action=recontestAssetsComparison
Winners /andhra2014/index.php?action=show_winners&sort=default
All candidates  /andhra2014/
Recontest /andhra2014/index.php?action=recontestAssetsComparison
Winners /ap09/index.php?action=show_winners&sort=default
All candidates  /ap09/
All candidates  /andhraPradesh2004/
Andhra Pradesh  done
Winners /ArunachalPradesh2019/index.php?action=show_winners&sort=default
All candidates  /ArunachalPradesh2019/
Recontest /ArunachalPradesh2019/index.php?action=recontestAssetsComparison
Winners /arunachal2014/index.php?action=show_winners&sort=default
All candidates  /arunachal2014/
Recontest /arunachal2014/index.php?action=recontestAssetsComparison
Winners /an2009/index.php?action=show_winners&sort=default
All candidates  /an2009/
Recontest /an2009/index.php?action=recontestAssetsComparison
Winners /an2004/index.php?acti

All candidates  /manipur2012/
Recontest /manipur2012/index.php?action=recontestAssetsComparison
Winners /manipur07/index.php?action=show_winners&sort=default
All candidates  /manipur07/
Manipur  done
Winners /meghalaya2018/index.php?action=show_winners&sort=default
All candidates  /meghalaya2018/
Recontest /meghalaya2018/index.php?action=recontestAssetsComparison
Winners /meghalaya2013/index.php?action=show_winners&sort=default
All candidates  /meghalaya2013/
Recontest /meghalaya2013/index.php?action=recontestAssetsComparison
Winners /2008Meghalaya/index.php?action=show_winners&sort=default
All candidates  /2008Meghalaya/
Meghalaya  done
Winners /mizoram2018/index.php?action=show_winners&sort=default
All candidates  /mizoram2018/
Recontest /mizoram2018/index.php?action=recontestAssetsComparison
Winners /mizoram2013/index.php?action=show_winners&sort=default
All candidates  /mizoram2013/
Recontest /mizoram2013/index.php?action=recontestAssetsComparison
Winners /2008Mizoram/index.php?act

In [22]:
df1 = pd.DataFrame.from_dict(dic1).T
df1

,state,year,Liabilities,candidate_id,Candidate,Constituency,Party,Criminal Case,Education,Total Assets
0,Andhra Pradesh,/andhrapradesh2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Andhra Pradesh,/andhrapradesh2019,"Rs 12,24,85,291 ~ 12 Crore+",5410,Cherukuvada Sriranganadha Raju,ACHANTA,YSRCP,0,8th Pass,"Rs 44,18,37,720 ~ 44 Crore+"
2,Andhra Pradesh,/andhrapradesh2019,"Rs 13,98,51,224 ~ 13 Crore+",5571,Gottipati Ravi Kumar,ADDANKI,TDP,1,12th Pass,"Rs 58,37,67,374 ~ 58 Crore+"
3,Andhra Pradesh,/andhrapradesh2019,"Rs 1,01,13,204 ~ 1 Crore+",5004,Y. Sai Prasad Reddy,ADONI,YSRCP,0,10th Pass,"Rs 9,32,23,587 ~ 9 Crore+"
4,Andhra Pradesh,/andhrapradesh2019,"Rs 75,29,752 ~ 75 Lacs+",4971,Brijendra Reddy Gangula\t,ALLAGADDA,YSRCP,1,Graduate,"Rs 1,71,32,578 ~ 1 Crore+"
...,...,...,...,...,...,...,...,...,...,...
13751,West Bengal,/wb2006,Rs 0 ~,215,MOHAN MONDAL,Uluberia North,CPM,0,Graduate,"Rs 2,05,648 ~ 2 Lacs+"
13752,West Bengal,/wb2006,Rs 0 ~,213,RABINDRA GHOSH,Uluberia South,AIFB,0,10th Pass,"Rs 20,82,068 ~ 20 Lacs+"
13753,West Bengal,/wb2006,Rs 0 ~,197,PROF. DR. SRUTINATH PRAHARAJ,Uttarpara,CPM,0,Doctorate,"Rs 13,94,612 ~ 13 Lacs+"
13754,West Bengal,/wb2006,Rs 0 ~,247,ANADI KUMAR SAHU,Vidyasagar,CPM,0,Graduate,"Rs 20,25,220 ~ 20 Lacs+"


In [23]:
df1.to_csv(r"C:\Users\nikhi\Desktop\ADR Politicians\winners_states.csv")

### Scrape data on LS election candidates

In [73]:
columns = ['Name', 'PAN Given(Y or N)', 'Age', 'Gender', 'Party Code', 'Criminal Cases', 'Number of Cases', 
           'Serious IPC Counts', 'IPC Details*', 'Education Level', 'Movable Assets', 
           'Immovable Assets', 'Total Assets', 'Total Liabilities']
num = len(columns)
dic2 = {}
count = 0
for link in soup.findAll('a'):
    url = link.get('href').replace(' ', '%20')
    if '/loksabha' not in url.lower() or 'ls in url.lower()':
        continue
    if 'winners' in url.lower():
        print("Winners", url)
        continue
    if 'recontestAssetsComparison' in url:
        print("Recontest", url)
        continue
    print(url)
    html = urlopen(url)
    soup1 = BeautifulSoup(html, 'lxml')
    for l in soup1.findAll('a'):
        u = l.get('href', None)
        if u is None:
            continue
        if 'state_id' in u:
            state = l.text
            state_id = u[u.rfind('=')+1:]
        if 'constituency_id' in u:
            constituency = l.text
            constituency_id = u[u.rfind('=')+1:]
            html = urlopen(url + u)
            soup2 = BeautifulSoup(html, 'lxml')
            for it in soup2.findAll('a'):
                t = str(it.get('href'))
                if 'comparisonchart' not in t:
                    continue
                html = urlopen(url + t)
                soup4 = BeautifulSoup(html, 'lxml')
                table_container = soup4.find_all('table')
                soup5 = BeautifulSoup(str(table_container), 'html.parser')
                first = 1
                for each in soup5.findAll('tr'):
                    if first == 1:
                        first = 0
                        continue
                    else:
                        c = 1
                        name = each.find('a').text
                        name_link = each.find('a').get('href')
                        name_id = name_link[name_link.rfind('=')+1:]
                        dic2[count] = {'state': state, 'constituency': constituency, 
                                        'constituency_id': constituency_id, 'candidate': name, 
                                        'candidate_id': name_id, 'year': url[url.find('info/')+5:] }
                        for content in each.findAll('td'):
                            dic2[count][columns[c]] = content.text
                            c+=1
                            if c == num:
                                c = 1
                        count+=1
        print(state, " done")

https://myneta.info/LokSabha2019/
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDAMAN & NICOBAR ISLANDS   done
ANDAMAN & NICOBAR ISLANDS   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   done
ANDHRA PRADESH   don

TAMIL NADU   done
TAMIL NADU   done
TAMIL NADU   done
TAMIL NADU   done
TAMIL NADU   done
TAMIL NADU   done
TAMIL NADU   done
TAMIL NADU   done
TAMIL NADU   done
TAMIL NADU   done
TAMIL NADU   done
TELANGANA   done
TELANGANA   done
TELANGANA   done
TELANGANA   done
TELANGANA   done
TELANGANA   done
TELANGANA   done
TELANGANA   done
TELANGANA   done
TELANGANA   done
TELANGANA   done
TELANGANA   done
TELANGANA   done
TELANGANA   done
TELANGANA   done
TELANGANA   done
TELANGANA   done
TELANGANA   done
TRIPURA   done
TRIPURA   done
TRIPURA   done
UTTAR PRADESH   done
UTTAR PRADESH   done
UTTAR PRADESH   done
UTTAR PRADESH   done
UTTAR PRADESH   done
UTTAR PRADESH   done
UTTAR PRADESH   done
UTTAR PRADESH   done
UTTAR PRADESH   done
UTTAR PRADESH   done
UTTAR PRADESH   done
UTTAR PRADESH   done
UTTAR PRADESH   done
UTTAR PRADESH   done
UTTAR PRADESH   done
UTTAR PRADESH   done
UTTAR PRADESH   done
UTTAR PRADESH   done
UTTAR PRADESH   done
UTTAR PRADESH   done
UTTAR PRADESH   done
UTTAR PRAD

MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MADHYA PRADESH   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA   done
MAHARASHTRA  

In [74]:
df2 = pd.DataFrame.from_dict(dic2).T
df2

,state,constituency,constituency_id,candidate,candidate_id,year,PAN Given(Y or N),Age,Gender,Party Code,Criminal Cases,Number of Cases,Serious IPC Counts,IPC Details*,Education Level,Movable Assets,Immovable Assets,Total Assets,Total Liabilities
0,ANDAMAN & NICOBAR ISLANDS,ANDAMAN AND NICOBAR ISLANDS,431,Ayan Mandal,4954,LokSabha2019/,Y,30,M,AITC,No,0,0,,Graduate,"8,02,940~ 8 Lacs+","64,67,500~ 64 Lacs+","72,70,440~ 72 Lacs+","15,00,000~ 15 Lacs+"
1,ANDAMAN & NICOBAR ISLANDS,ANDAMAN AND NICOBAR ISLANDS,431,C G Saji Kumar,5456,LokSabha2019/,Y,48,M,All India Hindustan Congress Party,No,0,0,,12th Pass,"1,20,000~ 1 Lacs+",0~,"1,20,000~ 1 Lacs+",0~
2,ANDAMAN & NICOBAR ISLANDS,ANDAMAN AND NICOBAR ISLANDS,431,C U Rasheed,5716,LokSabha2019/,Y,34,M,IND,No,0,0,,12th Pass,"1,22,808~ 1 Lacs+","80,000~ 80 Thou+","2,02,808~ 2 Lacs+","17,00,000~ 17 Lacs+"
3,ANDAMAN & NICOBAR ISLANDS,ANDAMAN AND NICOBAR ISLANDS,431,Gour Chandra Majumder,7201,LokSabha2019/,Y,52,M,IND,No,0,0,,Graduate,"62,000~ 62 Thou+","60,00,000~ 60 Lacs+","60,62,000~ 60 Lacs+",0~
4,ANDAMAN & NICOBAR ISLANDS,ANDAMAN AND NICOBAR ISLANDS,431,Henry,7203,LokSabha2019/,Y,50,M,IND,No,0,0,,10th Pass,"56,459~ 56 Thou+",0~,"56,459~ 56 Thou+",0~
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11617,WEST BENGAL,Vishnupur,537,Achintya Majhi,5433,loksabha2004/,N,38,M,INC,No,0,0,,Graduate,"67,264~ 67 Thou+",0~,"67,264~ 67 Thou+","50,050~ 50 Thou+"
11618,WEST BENGAL,Vishnupur,537,Ajoy Bauri,5435,loksabha2004/,N,40,M,BSP,No,0,0,,Not Given,"35,000~ 35 Thou+",0~,"35,000~ 35 Thou+",0~
11619,WEST BENGAL,Vishnupur,537,Janardan Saha,5432,loksabha2004/,N,37,M,AITC,No,0,0,,Post Graduate,"1,33,612~ 1 Lacs+","1,00,000~ 1 Lacs+","2,33,612~ 2 Lacs+","14,000~ 14 Thou+"
11620,WEST BENGAL,Vishnupur,537,Sraban Mondal,5434,loksabha2004/,N,40,M,IND,No,0,0,,Graduate,0~,0~,0~,0~


In [75]:
df2.to_csv(r"C:\Users\nikhi\Desktop\ADR Politicians\candidates_LS_elections_2004_19.csv")

### Scrape data on winners

In [76]:
columns = ['Liabilities', 'Candidate', 'Constituency', 'Party', 'Criminal Case', 'Education', 'Total Assets']
num = len(columns)
dic3 = {}
count = 0
for link in soup.findAll('a'):
    state = link.text
    url = link.get('href').replace(' ', '%20')
    if 'loksabha' not in url.lower() or 'ls' in url.lower():
        continue
    if 'expense' in url.lower():
        continue
    if 'winner' not in url.lower():
        continue
    print(url)
    html = urlopen(url)
    soup1 = BeautifulSoup(html, 'lxml')
    try:
        table_container = soup1.find_all('table')[2]
    except:
        continue
    soup2 = BeautifulSoup(str(table_container), 'html.parser')
    first = 1
    for each in soup2.findAll('tr'):
        if first == 1:
            first = 0
            continue
        else:
            dic3[count] = {'state': state, 'year': url[:url.rfind('/')] }
            c = 0
            for content in each.findAll('td'):
                if content.find('a') is not None:
                    name_link = content.find('a').get('href')
                    name_id = name_link[name_link.rfind('=')+1:]
                    dic3[count]['candidate_id'] = name_id
                dic3[count][columns[c]] = content.text
                c+=1
                if c == num:
                    c = 0
            count+=1
    print(state, " done")

https://myneta.info/LokSabha2019/index.php?action=show_winners&sort=default
 Winners  done
https://myneta.info/loksabha2004/index.php?action=show_winners&sort=default
 Winners  done


In [77]:
df3 = pd.DataFrame.from_dict(dic3).T
df3

,state,year,Liabilities,candidate_id,Candidate,Constituency,Party,Criminal Case,Education,Total Assets
0,Winners,https://myneta.info/LokSabha2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Winners,https://myneta.info/LokSabha2019,"Rs 2,31,450 ~ 2 Lacs+",5069,Bapu Rao Soyam,ADILABAD,BJP,52,12th Pass,"Rs 30,99,414 ~ 30 Lacs+"
2,Winners,https://myneta.info/LokSabha2019,"Rs 86,06,522 ~ 86 Lacs+",5522,Satyapal Singh Baghel,AGRA,BJP,5,Doctorate,"Rs 7,42,74,036 ~ 7 Crore+"
3,Winners,https://myneta.info/LokSabha2019,"Rs 62,52,577 ~ 62 Lacs+",9386,Patel Hasmukhbhai Somabhai,AHMEDABAD EAST,BJP,0,Others,"Rs 7,46,99,690 ~ 7 Crore+"
4,Winners,https://myneta.info/LokSabha2019,Rs 0 ~,8488,Dr. Solanki Kiritbhai,AHMEDABAD WEST,BJP,0,Post Graduate,"Rs 8,94,74,039 ~ 8 Crore+"
...,...,...,...,...,...,...,...,...,...,...
1053,Winners,https://myneta.info/loksabha2004,"Rs 11,212 ~ 11 Thou+",272,JANARDHANA REDDY NEDURUMALLI,Visakhapatnam,INC,0,Graduate Professional,"Rs 1,09,16,531 ~ 1 Crore+"
1054,Winners,https://myneta.info/loksabha2004,"Rs 10,000 ~ 10 Thou+",5431,Susmita Bauri,Vishnupur,CPM,0,Graduate Professional,"Rs 33,000 ~ 33 Thou+"
1055,Winners,https://myneta.info/loksabha2004,"Rs 4,47,393 ~ 4 Lacs+",2672,WAGMARE SURESH GANAPAT,Wardha,BJP,0,Graduate,"Rs 4,41,000 ~ 4 Lacs+"
1056,Winners,https://myneta.info/loksabha2004,"Rs 7,48,904 ~ 7 Lacs+",2679,GAWALI (PATIL) KU. BHAVANA PUNDLIKRAO,Washim,SHS,1,Graduate,"Rs 1,88,08,750 ~ 1 Crore+"


In [78]:
df3.to_csv(r"C:\Users\nikhi\Desktop\ADR Politicians\LS_winners_2004_19.csv")